# Genius


## Client application
code from https://www.johnwmillr.com/scraping-genius-lyrics/ and his designated [API](https://github.com/johnwmillr/lyricsgenius)

### Search song/artist


In [ ]:
# Sign up for a free account at Genius.com to access the API
# http://genius.com/api-clients
client_access_token = 'xVkIAGX-UFW9b49OybDSeoU1UTMyUBMlsuYe38ONfAiW0U54DM9qZx2UJu7VMpVi'

In [ ]:
# Let's take a look at how we might search for an artist using the Genius API.
import requests
import urllib2

# Format a request URL for the Genius API
search_term = 'Andy Shauf'
_URL_API = "https://api.genius.com/"
_URL_SEARCH = "search?q="
querystring = _URL_API + _URL_SEARCH + urllib2.quote(search_term)
request = urllib2.Request(querystring)
request.add_header("Authorization", "Bearer " + client_access_token)
# request.add_header("User-Agent","curl/7.9.8 (i686-pc-linux-gnu) libcurl 7.9.8 (OpnSSL 0.9.6b) (ipv6 enabled)")
request.add_header("User-Agent", "")

In [ ]:
# Now that we’ve formatted the URL, we can make a request to the database.
import json
response = urllib2.urlopen(request, timeout=3)
raw = response.read()
json_obj = json.loads(raw)

In [ ]:
# The 'hits` key stores info on each song in the search result.
# From here it's easy to grab the song title, album, etc.

# List each key contained within a single search hit
[key for key in json_obj['response']['hits'][0]['result']]

[u'song_art_image_thumbnail_url',
 u'api_path',
 u'stats',
 u'song_art_image_url',
 u'song_art_text_color',
 u'lyrics_state',
 u'title',
 u'pyongs_count',
 u'song_art_secondary_color',
 u'lyrics_owner_id',
 u'song_art_primary_color',
 u'url',
 u'full_title',
 u'title_with_featured',
 u'header_image_thumbnail_url',
 u'path',
 u'primary_artist',
 u'annotation_count',
 u'id',
 u'header_image_url']

### Scrape song lyrics ###

In [ ]:
from bs4 import BeautifulSoup
import re
URL = 'https://genius.com/Andy-shauf-the-magician-lyrics'
page = requests.get(URL)    
html = BeautifulSoup(page.text, "html.parser") # Extract the page's HTML as a string

# Scrape the song lyrics from the HTML
lyrics = html.find("div", class_="lyrics").get_text().encode('ascii','ignore')
# lyrics = re.sub('\[.*\]','',lyrics) # Remove [Verse] and [Bridge] stuff
# lyrics = re.sub('\n{2}','',lyrics)  # Remove gaps between verses        
# lyrics = str(lyrics).strip('\n')
print(lyrics[:150]+'...')



[Verse 1]
Do you find
It gets a little easier each time you make it disappear?
Oh fools, the magician bends the rules
As the crowd watches his every...


## Server application
use the [API](https://github.com/johnwmillr/lyricsgenius) `lyricsgenius/auth.py` 

In [ ]:
# check tests/test_auth.py for codes using auth.py

# Azlyrics


In [ ]:
import json
import requests
from bs4 import BeautifulSoup

agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
headers = {'User-Agent': agent}
base_url = "https://www.azlyrics.com"

In [ ]:
def _getArtists(letter):
  res = requests.get(f"{base_url}/{letter}.html", headers=headers)
  soup = BeautifulSoup(res.content, 'html.parser')
  artists = soup.find('div', attrs={'class': 'row'}).findAll('a')
  return {a.text.strip(): f"{base_url}/{a.get_attribute('href')}" for a in artists}

def _getSongs(artist_url):
  res = requests.get(artist_url, headers=headers)
  soup = BeautifulSoup(res.content, 'html.parser')
  songs = soup.findAll('div', attrs={'class': 'listalbum-item'})
  return {s.find('a').text.strip(): f"{base_url}/{s.find('a').get_attribute('href')}" for s in songs}

def _getLyrics(songs_url):
  res = requests.get(artist_url, headers=headers)
  soup = BeautifulSoup(res.content, 'html.parser')
  lyrics = soup.findAll('div', attrs={"class": None, "id": None})
  lyrics = [l.getText() for l in lyrics]
  return "\n".join(lyrics)

In [ ]:
all_artists = {}
# for letter in "abcdefghijklmnopqrstuvwxyz":
for letter in "ab":
    all_artists = {**all_artists, **_getArtists(letter)}
all_artists = {**all_artists, **_getArtists('19')}

pk = 0
formated_songs = []

for artist, artist_url in all_artists.items():
  for song, song_url in _getSongs(artist_url).items():
    formated_songs.append({
        'model': "searchApp.Song",
        'pk': pk,
        'fields': {
            'title': song,
            'artist': artist,
            'lyrics': _getLyrics(song_url),
        }
    })
    pk += 1
with open('songs.json', 'w') as f:
    json.dump(formated_songs, f)
formated_songs

# Lyrics.com


In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import time

agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
headers = {'User-Agent': agent}
base_url = "https://www.lyrics.com"

In [ ]:
base_url = "https://www.lyrics.com"

def _getArtists(letter):
    res = requests.get(f"{base_url}/artists/{letter.upper()}/99999", headers=headers)
    soup = BeautifulSoup(res.content, 'html.parser')
    artists = soup.find('div', attrs={'id': 'content-body'})
    if artists:
      artists = artists.findAll('strong')
      all_artists = {}
      for artist in artists:
          try:
            a = artist.find('a')
            all_artists = {**all_artists, **{a.text.strip(): f"{base_url}/{a.get('href')}"}}
          except Exception as e:
            print("Error in 'for artist in artists': ",e)
            print("sleeping for 1 min")
            time.sleep(60)
            pass
      return all_artists
    return {}

def _getSongs(artist_url):
    res = requests.get(artist_url, headers=headers)
    soup = BeautifulSoup(res.content, 'html.parser')
    songs = soup.find('div', attrs={'id': 'content-body'})
    all_songs = {}
    try:
      songs = songs.findAll('strong')
      for song in songs:
          try:
              s = song.find('a')
              all_songs = {**all_songs, **{ s.text.strip(): f"{base_url}/{s.get('href')}" }}
          except Exception as e:
            print("Error in 'for song in songs': ", e)
            print("sleeping for 1 min")
            time.sleep(60)
            pass
    except Exception as e:
      print("Error in 'getSongs': ", e)
      print("sleeping for 1 min")
      time.sleep(60)
    finally:
      return all_songs

def _getLyrics(song_url):
    res = requests.get(song_url, headers=headers)
    soup = BeautifulSoup(res.content, 'html.parser')
    lyrics = soup.find('pre', attrs={'id': 'lyric-body-text'})
    if lyrics:
        return lyrics.getText()
    return ""

# populateDatabase
def populateDB(letter):
    try:
      pk = 0
      with open(f'lyrics_{letter}_songs.txt', 'a+') as songfile:
          artists = _getArtists(letter)
          for artist, artist_url in artists.items():
              songs = _getSongs(artist_url)
              for song, song_url in songs.items():
                  s = {
                      'model': "searchApp.Song",
                      'pk': pk,
                      'fields': {
                          'title': song,
                          'artist': artist,
                          'lyrics': _getLyrics(song_url),
                      }
                  }
                  json.dump(s, songfile)
                  songfile.write("\n")

                  pk += 1
                  print(f"({letter.upper()}) {pk} songs added so far !")
    except Exception as e:
        print(repr(e))

In [ ]:
%%time
import concurrent.futures

try: 
  with concurrent.futures.ThreadPoolExecutor(max_workers=13) as executor:
      executor.map(populateDB, "0abcdefghijklmnopqrstuvwxyz")
except KeyboardInterrupt:
  pass

Streaming output truncated to the last 5000 lines.
(A) 2828 songs added so far !
(I) 2602 songs added so far !
(A) 2829 songs added so far !
(I) 2603 songs added so far !
(A) 2830 songs added so far !
(K) 2551 songs added so far !
(K) 2552 songs added so far !(A) 2831 songs added so far !

(I) 2604 songs added so far !
(H) 2315 songs added so far !
(K) 2553 songs added so far !
(A) 2832 songs added so far !
(I) 2605 songs added so far !
(H) 2316 songs added so far !
(K) 2554 songs added so far !
(A) 2833 songs added so far !
(K) 2555 songs added so far !
(H) 2317 songs added so far !
(I) 2606 songs added so far !
(A) 2834 songs added so far !
Error in 'for song in songs':  'NoneType' object has no attribute 'text'
sleeping for 1 min
(I) 2607 songs added so far !
(K) 2556 songs added so far !
(H) 2318 songs added so far !
(H) 2319 songs added so far !
(K) 2557 songs added so far !
(I) 2608 songs added so far !
(H) 2320 songs added so far !
(K) 2558 songs added so far !
(I) 2609 songs ad

KeyboardInterrupt: ignored

In [ ]:
# txt to json
import json
complete = []
pk = 0
with open('lyrics_songs.json', 'w') as g:
  for letter in "0abcdefghijklmnopqrstuvwxyz":
    with open(f'lyrics_{letter}_songs.txt', 'r') as f:
      for line in f:
        s = json.loads(line)
        s['pk'] = pk

        complete.append(s)
        pk += 1
  json.dump(complete, g)

JSONDecodeError: ignored

In [ ]:
import json
comlete = []
with open('lyrics_songs(multithreaded).txt', 'r') as f:
  with open('lyrics_songs.json', 'w') as g:
    for line in f:
      s = json.loads(line)
      complete.append(s)
    json.dump(complete, g)

In [ ]:
import pandas as pd
pd.read_json('lyrics_songs(multithreaded).txt', orient='records').to_json('lyrics_songs.json', orient='records')

In [ ]:
!zip lyrics.zip *.txt

(L) 3289 songs added so far !
  adding: lyrics_0_songs.txt(G) 2783 songs added so far !
(A) 3091 songs added so far !
(L) 3290 songs added so far !
 (deflated 74%)
  adding: lyrics_a_songs.txt (deflated 76%)
  adding: lyrics_b_songs.txt(G) 2784 songs added so far !
(L) 3291 songs added so far !
(A) 3092 songs added so far !
 (deflated 74%)
  adding: lyrics_c_songs.txt(L) 3292 songs added so far !
(G) 2785 songs added so far !
 (deflated 75%)
  adding: lyrics_d_songs.txt(A) 3093 songs added so far !
 (deflated 74%)
  adding: lyrics_e_songs.txt(L) 3293 songs added so far !
(G) 2786 songs added so far !
(A) 3094 songs added so far !
 (deflated 75%)
  adding: lyrics_f_songs.txt(L) 3294 songs added so far !
(G) 2787 songs added so far !
 (deflated 75%)
  adding: lyrics_g_songs.txt(A) 3095 songs added so far !

	zip warning:  file size changed while zipping lyrics_g_songs.txt
 (deflated 74%)
  adding: lyrics_h_songs.txt(L) 3295 songs added so far !
 (deflated 77%)
  adding: lyrics_i_songs.tx

# Format Datasets


## kaggle dataset


In [ ]:
!unzip "kaggle_dataset.zip"

In [ ]:
import pandas as pd
artists = pd.read_csv('artist_data.csv')[['Artist', 'Link']]
lyrics = pd.read_csv('lyrics_data.csv')[['ALink', 'SName', 'Lyric']]
df = lyrics.merge(artists, left_on='Link', right_on='ALink', how='inner')

df = df.rename(columns={'SName': 'title', 'Lyric': 'lyrics', 'Artist': 'artist'})[['title', 'artist', 'lyrics']]
df.to_json("kaggle_songs.json", orient='records')

## spotify billboard 2000-2018